In [150]:
import numpy as np
import qlbmlib
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt
import importlib
from qiskit_aer import StatevectorSimulator
from qiskit import transpile

importlib.reload(qlbmlib)

<module 'qlbmlib' from '/home/tavaa/research/qlbm/qlbmlib.py'>

## Configuration Parameters

Change these values to customize the simulation

In [151]:
# ============ CONFIGURATION ============
# Match Experiment2DCross setup

# Grid dimensions
GRID_SHAPE = (16, 16)  # 16x16 lattice

# Velocity links - D2Q9
LINKS = [[0,0], [1, 0], [0,1], [-1,0], [0,-1], [1,1], [-1,1], [-1,-1], [1,-1]]

# Velocity field preset ('zero' or 'vortex')
VELOCITY_PRESET = 'vortex'  # 'zero' or 'vortex'
VORTEX_MAGNITUDE = 0.5

# Simulation parameters
NUM_ITERATIONS = 5
SPEED_OF_SOUND = 1.0 / np.sqrt(3)

# ======================================

In [152]:
# ============ AUTO-GENERATED FROM CONFIG ============

def get_cross_initial_distribution(sites: tuple[int, int], 
                                 line_width: int = 2,
                                 background_density: float = 0.1,
                                 cross_density: float = 0.9):
    """Create a cross-shaped initial distribution."""
    density = np.full(sites, background_density)
    center_x = sites[0] // 2
    center_y = sites[1] // 2
    density[:, center_y-line_width:center_y+line_width] = cross_density
    density[center_x-line_width:center_x+line_width, :] = cross_density
    return density

def get_vortex_velocity_field(sites: tuple[int, int], magnitude: float = 0.2, clockwise: bool = True):
    """Create a vortex velocity field."""
    x = np.arange(sites[0])
    y = np.arange(sites[1])
    X, Y = np.meshgrid(x, y)
    
    center_x = sites[0] / 2
    center_y = sites[1] / 2
    X_centered = X.T - center_x
    Y_centered = Y.T - center_y
    
    r = np.sqrt(X_centered**2 + Y_centered**2) + 1e-6
    
    direction = 1 if clockwise else -1
    v_theta = magnitude * direction
    
    u = -v_theta * Y_centered / r
    v = v_theta * X_centered / r
    
    return np.stack([u, v])

# Create initial density with cross pattern
initial_density = get_cross_initial_distribution(GRID_SHAPE, line_width=2)

# Generate weights for D2Q9
weights = [4/9, 1/9, 1/9, 1/9, 1/9, 1/36, 1/36, 1/36, 1/36]
num_links = len(LINKS)

# Generate velocity field
if VELOCITY_PRESET == 'zero':
    velocity_field = np.zeros((2, GRID_SHAPE[0], GRID_SHAPE[1]))
elif VELOCITY_PRESET == 'vortex':
    velocity_field = get_vortex_velocity_field(GRID_SHAPE, magnitude=VORTEX_MAGNITUDE, clockwise=True)
else:
    raise ValueError(f"Unknown velocity preset: {VELOCITY_PRESET}")

# Calculate derived parameters
site_qubits_per_dim = [int(np.ceil(np.log2(dim))) for dim in GRID_SHAPE]
site_qubits = sum(site_qubits_per_dim)
link_qubits = int(np.ceil(np.log2(num_links)))
collision_matrix = qlbmlib.get_collision_diagonal(
    link_qubits, LINKS, weights, velocity_field, SPEED_OF_SOUND
)

print("Configuration:")
print(f"  Grid: {GRID_SHAPE}")
print(f"  Total sites: {np.prod(GRID_SHAPE)}")
print(f"  Links: {num_links} ({link_qubits} qubits)")
print(f"  Site qubits: {site_qubits} (per dim: {site_qubits_per_dim})")
print(f"  Velocity preset: {VELOCITY_PRESET}")
print(f"\nInitial density sum: {np.sum(initial_density):.6f}")

Configuration:
  Grid: (16, 16)
  Total sites: 256
  Links: 9 (4 qubits)
  Site qubits: 8 (per dim: [4, 4])
  Velocity preset: vortex

Initial density sum: 115.200000


In [153]:
# Compare one iteration: circuit collision vs manual diagonal matrix multiplication
from qiskit import QuantumCircuit

# Reset to initial state
test_density = initial_density.copy()
norm = np.linalg.norm(test_density)

# Encode the state
state = qlbmlib.encode(test_density, link_qubits)

# Initialize circuit with state
qc = QuantumCircuit(state.num_qubits)
qc.initialize(state)

# Apply link encoding
qc.append(qlbmlib.encode_links(link_qubits, num_links), list(range(site_qubits, state.num_qubits - 1)))
state_after_encoding = state.evolve(qc)

print("State after encoding (first 20 amplitudes):")
state_array = np.array(state_after_encoding)
for i in range(min(20, len(state_array))):
    if np.abs(state_array[i]) > 1e-10:
        print(f"  [{i:4d}] = {state_array[i].real:+.6f} + {state_array[i].imag:+.6f}i")

# Method 1: Apply collision via circuit
collision_circuit = qlbmlib.collision_nonuniform(site_qubits, link_qubits, collision_matrix)
state_circuit = state_after_encoding.evolve(collision_circuit, list(range(state.num_qubits)))

print("\n=== Method 1: Circuit Collision ===")
print("State after circuit collision (first 20 amplitudes):")
state_circuit_array = np.array(state_circuit)
for i in range(min(20, len(state_circuit_array))):
    if np.abs(state_circuit_array[i]) > 1e-10:
        print(f"  [{i:4d}] = {state_circuit_array[i].real:+.6f} + {state_circuit_array[i].imag:+.6f}i")

# Method 2: Manual diagonal matrix multiplication
print("\n=== Method 2: Manual Diagonal Multiplication ===")

# Get the state vector as array
state_vec = np.array(state_after_encoding)
num_target_qubits = site_qubits + link_qubits
target_dim = 2**num_target_qubits

# Create the full diagonal matrix for the collision step
# The collision gate is controlled by the ancilla qubit
# When ancilla = 0: apply (c + i*sqrt(1-c^2))
# When ancilla = 1: apply (c - i*sqrt(1-c^2))

diagonal_0 = collision_matrix + 1.j * np.sqrt(1 - np.square(collision_matrix))
diagonal_1 = collision_matrix - 1.j * np.sqrt(1 - np.square(collision_matrix))

# Full state has 2^(num_qubits) components
# Ancilla is the last qubit
# When ancilla = 0: indices 0 to 2^(num_qubits-1) - 1
# When ancilla = 1: indices 2^(num_qubits-1) to 2^num_qubits - 1

full_diagonal = np.zeros(len(state_vec), dtype=complex)
half_size = len(state_vec) // 2

# Note: Hadamard on ancilla creates superposition, so we need to account for that
# H creates (|0⟩ + |1⟩)/sqrt(2), so we apply the transformation in the Hadamard basis

# Actually, the collision gate structure is:
# H ⊗ controlled_diag(0) ⊗ controlled_diag(1) ⊗ H
# This is equivalent to a diagonal in the computational basis

# For simplicity, let's just extract what the diagonal should be after H ⊗ D ⊗ H
# Since the ancilla starts in |0⟩ and gets Hadamard, we get |+⟩
# The controlled diagonals act based on ancilla state
# Then another Hadamard brings it back

# The net effect is: for each computational basis state |x,a⟩:
# If a=0: multiply by (diagonal_0[x] + diagonal_1[x])/2
# If a=1: multiply by (diagonal_0[x] - diagonal_1[x])/2

# But actually, simpler: just apply the circuit and compare
# Let's compute what the diagonal should be directly

# Apply H, then controlled diagonals, then H on ancilla
# This gives a diagonal operation in the original basis

for i in range(len(state_vec)):
    # Determine target index (all qubits except ancilla)
    target_idx = i % target_dim
    ancilla_bit = i // target_dim
    
    if ancilla_bit == 0:
        full_diagonal[i] = diagonal_0[target_idx]
    else:
        full_diagonal[i] = diagonal_1[target_idx]

# Apply manual multiplication (but need to account for Hadamard on ancilla)
# The structure is H ⊗ Controlled-Diag ⊗ H on ancilla
# Let's manually compute the effect

state_manual = state_vec.copy()

# Apply first Hadamard on ancilla (last qubit)
state_manual_reshaped = state_manual.reshape(2, -1)
state_after_H1 = np.zeros_like(state_manual_reshaped, dtype=complex)
state_after_H1[0] = (state_manual_reshaped[0] + state_manual_reshaped[1]) / np.sqrt(2)
state_after_H1[1] = (state_manual_reshaped[0] - state_manual_reshaped[1]) / np.sqrt(2)

# Apply diagonal gates
state_after_diag = state_after_H1.copy()
state_after_diag[0] *= diagonal_0
state_after_diag[1] *= diagonal_1

# Apply second Hadamard on ancilla
state_manual_reshaped = np.zeros_like(state_after_diag, dtype=complex)
state_manual_reshaped[0] = (state_after_diag[0] + state_after_diag[1]) / np.sqrt(2)
state_manual_reshaped[1] = (state_after_diag[0] - state_after_diag[1]) / np.sqrt(2)

state_manual_final = state_manual_reshaped.flatten()

print("State after manual collision (first 20 amplitudes):")
for i in range(min(20, len(state_manual_final))):
    if np.abs(state_manual_final[i]) > 1e-10:
        print(f"  [{i:4d}] = {state_manual_final[i].real:+.6f} + {state_manual_final[i].imag:+.6f}i")

# Compare the two methods
print("\n=== Comparison ===")
difference = state_circuit_array - state_manual_final
max_diff = np.max(np.abs(difference))
print(f"Max absolute difference: {max_diff:.2e}")
print(f"Are they equal (within 1e-10)? {max_diff < 1e-10}")

if max_diff > 1e-10:
    print("\nLargest differences:")
    sorted_indices = np.argsort(np.abs(difference))[::-1]
    for idx in sorted_indices[:10]:
        if np.abs(difference[idx]) > 1e-10:
            print(f"  [{idx:4d}] Circuit: {state_circuit_array[idx]:.6f}, Manual: {state_manual_final[idx]:.6f}, Diff: {difference[idx]:.6f}")

State after encoding (first 20 amplitudes):
  [   0] = +0.002604 + +0.000000i
  [   1] = +0.002604 + +0.000000i
  [   2] = +0.002604 + +0.000000i
  [   3] = +0.002604 + +0.000000i
  [   4] = +0.002604 + +0.000000i
  [   5] = +0.002604 + +0.000000i
  [   6] = +0.023437 + +0.000000i
  [   7] = +0.023437 + +0.000000i
  [   8] = +0.023437 + +0.000000i
  [   9] = +0.023437 + +0.000000i
  [  10] = +0.002604 + +0.000000i
  [  11] = +0.002604 + +0.000000i
  [  12] = +0.002604 + +0.000000i
  [  13] = +0.002604 + +0.000000i
  [  14] = +0.002604 + +0.000000i
  [  15] = +0.002604 + +0.000000i
  [  16] = +0.002604 + +0.000000i
  [  17] = +0.002604 + +0.000000i
  [  18] = +0.002604 + +0.000000i
  [  19] = +0.002604 + +0.000000i

=== Method 1: Circuit Collision ===
State after circuit collision (first 20 amplitudes):
  [   0] = +0.001157 + -0.000000i
  [   1] = +0.001157 + -0.000000i
  [   2] = +0.001157 + +0.000000i
  [   3] = +0.001157 + -0.000000i
  [   4] = +0.001157 + -0.000000i
  [   5] = +0.00

## Compare Circuit vs Manual Diagonal Multiplication

In [161]:
# Compare: Full circuit iteration vs just multiplying by collision diagonal
from qiskit import QuantumCircuit

# Reset to initial state
test_density = initial_density.copy()
norm = np.linalg.norm(test_density)

print("=== Method 1: Full Circuit (with Hadamard/Control gates) ===")
# Encode the state
state = qlbmlib.encode(test_density, link_qubits)
print(f"Initial state dimension: {len(state)}")
print(f"Total qubits: {state.num_qubits}")

# Initialize circuit with state
qc = QuantumCircuit(state.num_qubits)
qc.initialize(state)

# Apply link encoding
qc.append(qlbmlib.encode_links(link_qubits, num_links), list(range(site_qubits, state.num_qubits - 1)))
state_after_encoding = state.evolve(qc)

# Apply collision via circuit (with Hadamard and controlled gates)
collision_circuit = qlbmlib.collision_nonuniform(site_qubits, link_qubits, collision_matrix)
state_circuit = state_after_encoding.evolve(collision_circuit, list(range(state.num_qubits)))

state_circuit_array = np.array(state_circuit)

print("\n=== Method 2: Direct Diagonal Multiplication (No Hadamard/Control) ===")
# Just multiply the relevant part of the state by the diagonal

# Start with state after encoding
state_vec = np.array(state_after_encoding).copy()
num_target_qubits = site_qubits + link_qubits
target_dim = 2**num_target_qubits

print(f"Collision matrix dimension: {len(collision_matrix)}")
print(f"Target qubits: {num_target_qubits}, Target dim: {target_dim}")

# The state has structure: |site⟩|link⟩|ancilla⟩
# We want to multiply only the |site⟩|link⟩ part by the collision diagonal
# The ancilla should remain unchanged

# Reshape to separate ancilla from the rest
state_reshaped = state_vec.reshape(2, -1)  # [ancilla_bit][other_qubits]

print(f"State shape after reshape: {state_reshaped.shape}")
print(f"Expected: (2, {target_dim})")

# Apply diagonal multiplication to both ancilla states
state_direct = state_reshaped.copy()
for ancilla_val in range(2):
    # Multiply each component by the corresponding diagonal element
    state_direct[ancilla_val] *= collision_matrix

state_direct_final = state_direct.flatten()

print("\n=== Extract Distribution Elements Only ===")
# Only compare elements corresponding to actual velocity links (not residual subspace)
num_sites = np.prod(GRID_SHAPE)

# For ancilla=0, extract only the first num_links * num_sites elements
# The state is organized as |link⟩|site⟩|ancilla⟩ (after encoding)
# So we need to extract link indices 0 to num_links-1

distribution_indices = []
for link_idx in range(num_links):
    for site_idx in range(num_sites):
        # Index in the flattened state
        # Structure: [ancilla][link * num_sites + site]
        idx_ancilla_0 = 0 * target_dim + link_idx * num_sites + site_idx
        distribution_indices.append(idx_ancilla_0)

print(f"Number of distribution elements: {len(distribution_indices)}")
print(f"Expected: {num_links * num_sites}")

# Extract only distribution elements from both states
circuit_distribution = state_circuit_array[distribution_indices]
direct_distribution = state_direct_final[distribution_indices]

print(f"\nCircuit distribution (first 20 non-zero elements):")
count = 0
for i, idx in enumerate(distribution_indices):
    if np.abs(circuit_distribution[i]) > 1e-10:
        print(f"  [idx={idx:4d}] = {circuit_distribution[i].real:+.8f} + {circuit_distribution[i].imag:+.8f}i  |amp|={np.abs(circuit_distribution[i]):.8f}")
        count += 1
        if count >= 20:
            break

print(f"\nDirect distribution (first 20 non-zero elements):")
count = 0
for i, idx in enumerate(distribution_indices):
    if np.abs(direct_distribution[i]) > 1e-10:
        print(f"  [idx={idx:4d}] = {direct_distribution[i].real:+.8f} + {direct_distribution[i].imag:+.8f}i  |amp|={np.abs(direct_distribution[i]):.8f}")
        count += 1
        if count >= 20:
            break

print("\n=== Comparison (Distribution Elements Only) ===")
difference = circuit_distribution - direct_distribution
max_diff = np.max(np.abs(difference))
print(f"Max absolute difference: {max_diff:.3e}")
print(f"RMS difference: {np.sqrt(np.mean(np.abs(difference)**2)):.3e}")
print(f"Are they equal (within 1e-10)? {max_diff < 1e-10}")

if max_diff > 1e-10:
    print("\nLargest differences (showing top 10):")
    sorted_indices = np.argsort(np.abs(difference))[::-1]
    for i in sorted_indices[:10]:
        if np.abs(difference[i]) > 1e-10:
            idx = distribution_indices[i]
            print(f"  [idx={idx:4d}] Circuit: {circuit_distribution[i].real:+.8f}{circuit_distribution[i].imag:+.8f}i, "
                  f"Direct: {direct_distribution[i].real:+.8f}{direct_distribution[i].imag:+.8f}i, "
                  f"Diff: {np.abs(difference[i]):.3e}")
            
# Also compare norms
norm_circuit = np.linalg.norm(circuit_distribution)
norm_direct = np.linalg.norm(direct_distribution)
print(f"\nNorm comparison (distribution elements only):")
print(f"  Circuit norm: {norm_circuit:.10f}")
print(f"  Direct norm: {norm_direct:.10f}")
print(f"  Difference: {abs(norm_circuit - norm_direct):.3e}")

# Check imaginary parts
max_imag_circuit = np.max(np.abs(circuit_distribution.imag))
max_imag_direct = np.max(np.abs(direct_distribution.imag))
print(f"\nImaginary part check:")
print(f"  Circuit max |imag|: {max_imag_circuit:.3e}")
print(f"  Direct max |imag|: {max_imag_direct:.3e}")

=== Method 1: Full Circuit (with Hadamard/Control gates) ===
Initial state dimension: 8192
Total qubits: 13

=== Method 2: Direct Diagonal Multiplication (No Hadamard/Control) ===
Collision matrix dimension: 4096
Target qubits: 12, Target dim: 4096
State shape after reshape: (2, 4096)
Expected: (2, 4096)

=== Extract Distribution Elements Only ===
Number of distribution elements: 2304
Expected: 2304

Circuit distribution (first 20 non-zero elements):
  [idx=   0] = +0.00115741 + -0.00000000i  |amp|=0.00115741
  [idx=   1] = +0.00115741 + -0.00000000i  |amp|=0.00115741
  [idx=   2] = +0.00115741 + +0.00000000i  |amp|=0.00115741
  [idx=   3] = +0.00115741 + -0.00000000i  |amp|=0.00115741
  [idx=   4] = +0.00115741 + -0.00000000i  |amp|=0.00115741
  [idx=   5] = +0.00115741 + -0.00000000i  |amp|=0.00115741
  [idx=   6] = +0.01041667 + -0.00000000i  |amp|=0.01041667
  [idx=   7] = +0.01041667 + -0.00000000i  |amp|=0.01041667
  [idx=   8] = +0.01041667 + -0.00000000i  |amp|=0.01041667
  [id